## 1. Importing Necessary Libraries

In [156]:
from pathlib import Path

import numpy as np
import pandas as pd

import sklearn
import xgboost

from sklearn.metrics import r2_score,mean_absolute_error,root_mean_squared_error

from sklearn.linear_model import (
      LinearRegression,
      Ridge,
      Lasso
)
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import (
      RandomForestRegressor,
      AdaBoostRegressor,
      GradientBoostingRegressor
)
from xgboost import XGBRegressor


from sklearn.base import (
    BaseEstimator,
    TransformerMixin
)

from sklearn.impute import (
    SimpleImputer
)

from sklearn.preprocessing import (
	OneHotEncoder,
    OrdinalEncoder,
    TargetEncoder,
    StandardScaler,
    MinMaxScaler,
    FunctionTransformer
)

from sklearn.compose import (
    ColumnTransformer,
    TransformedTargetRegressor
)

from sklearn.pipeline import (
    Pipeline,
    FeatureUnion
)

from feature_engine.encoding import(
    RareLabelEncoder
)

from feature_engine.selection import(
    SelectBySingleFeaturePerformance,
    SmartCorrelatedSelection
)

import matplotlib.pyplot as plt

import optuna

import mlflow

import warnings

## 2. Display settings

In [4]:
pd.set_option("display.max_columns", None)
sklearn.set_config(transform_output = "pandas")
warnings.filterwarnings('ignore')

## 3. Reading The Data

In [6]:
PROJECT_DIR = Path(r"F:\Rishabh\House-Price-Prediction-MLOps-Project")
DATA_DIR = "data"

In [7]:
def read_data(name):
    file_name = f"{name}.csv"
    file_path = PROJECT_DIR/DATA_DIR/name/file_name

    return pd.read_csv(file_path)

In [8]:
train = read_data("train")

In [9]:
val = read_data("val")

In [10]:
test = read_data("test")

In [11]:
X_train = train.drop(columns = ["amount"])
y_train = train.amount.copy()

X_val = val.drop(columns = ["amount"])
y_val = val.amount.copy()

X_test = test.drop(columns = ["amount"])
y_test = test.amount.copy()

In [12]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

(42129, 18)
(42129,)
(4681, 18)
(4681,)
(5202, 18)
(5202,)


## 4. Feature Engineering Pipeline

In [80]:
# Function to rename the columns before putting back into imputation pipeline
def prefix_remover(X, prefixes):

    prefix_list = [f"{prefix}__" for prefix in prefixes]
    new_cols = X.columns
    
    for prefix in prefix_list:
        new_cols = [col.replace(prefix,"") if col.startswith(prefix) else col for col in new_cols]

    return X.rename(
        columns = dict(zip(X.columns,new_cols))
    )

# imputer to impute bathroom values based on num_bhk
class GroupAggregateImputer(BaseEstimator, TransformerMixin):

    def __init__(self,variable,group_col,estimator,add_indicator = False):
        self.variable = variable
        self.group_col = group_col
        self.estimator = estimator
        self.add_indicator = add_indicator

    def fit(self,X,y = None):

        self.group_medians_ = {}
        self.group_modes_ = {}

        if self.estimator == "median":
                self.group_medians_[self.variable] =  X.groupby(self.group_col)[self.variable].median()
                
        elif self.estimator == "mode":
                self.group_modes_[self.variable] =  X.groupby(self.group_col)[self.variable].agg(lambda x: x.mode().iloc[0])
            
        return self

    def transform(self,X):
       X = X.copy()

       if self.add_indicator:
           X = X.assign(**{
               f"{self.variable}_missingindicator" : lambda df:(
                     np.where(
                         df[self.variable].isnull(),
                         1,0
                     )
                 )
           })

       if self.estimator == "median":
               mask = X[self.variable].isnull()
               X.loc[mask,self.variable] = X.loc[mask,self.group_col].map(self.group_medians_[self.variable])
               
       elif self.estimator == "mode":
               mask = X[self.variable].isnull()
               X.loc[mask,self.variable] = X.loc[mask,self.group_col].map(self.group_modes_[self.variable]) 

       
       return X

# Imputation pipeline

furnishing_imputer = ColumnTransformer(transformers = [
    ("furnishing_imputer",GroupAggregateImputer(variable = "furnishing",group_col = "transaction",estimator = "mode"),["furnishing","transaction"])
],remainder = "passthrough")

furnishing_imputation_pipeline = Pipeline(steps = [
    ("furnishing_imputer",furnishing_imputer),
    ("prefix_remover",FunctionTransformer(func = prefix_remover, kw_args = {"prefixes" : ["furnishing_imputer","remainder"]}))
])

floor_num_imputer = ColumnTransformer(transformers = [
    ("floor_num_imputer",SimpleImputer(strategy = "median"),["floor_num"])
],remainder = "passthrough")

floor_num_imputation_pipeline = Pipeline(steps = [
    ("floor_num_imputer",floor_num_imputer),
    ("prefix_remover",FunctionTransformer(func = prefix_remover, kw_args = {"prefixes" : ["floor_num_imputer","remainder"]}))
])

num_floors_imputer = ColumnTransformer(transformers = [
    ("num_floors_imputer",GroupAggregateImputer(variable = "num_floors",group_col = "floor_num",estimator = "median"),["num_floors","floor_num"])
],remainder = "passthrough")

num_floors_imputation_pipeline = Pipeline(steps = [
    ("num_floors_imputer",num_floors_imputer),
    ("prefix_remover",FunctionTransformer(func = prefix_remover, kw_args = {"prefixes" : ["num_floors_imputer","remainder"]}))
])

balcony_imputer = ColumnTransformer(transformers = [
    ("balcony_imputer", GroupAggregateImputer(variable = "balcony",group_col = "num_bhk",estimator = "median", add_indicator = True),["balcony","num_bhk"])
],remainder = "passthrough")

balcony_imputation_pipeline = Pipeline(steps = [
    ("balcony_imputer",balcony_imputer),
    ("prefix_remover",FunctionTransformer(func = prefix_remover, kw_args = {"prefixes" : ["balcony_imputer","remainder"]}))
])

ownership_imputer = ColumnTransformer(transformers = [
    ("ownership_imputer",SimpleImputer(strategy = 'most_frequent',add_indicator = True),["ownership"])
], remainder = "passthrough")


ownership_imputation_pipeline = Pipeline(steps = [
    ("ownership_imputer",ownership_imputer),
    ("prefix_remover",FunctionTransformer(func = prefix_remover, kw_args = {"prefixes" : ["ownership_imputer","remainder"]}))
])

facing_imputer = ColumnTransformer(transformers = [
    ("facing_imputer",SimpleImputer(strategy = 'constant',fill_value = 'Missing',add_indicator = True),["facing"])
], remainder = "passthrough")

facing_imputation_pipeline = Pipeline(steps = [
    ("facing_imputer",facing_imputer),
    ("prefix_remover",FunctionTransformer(func = prefix_remover, kw_args = {"prefixes" : ["facing_imputer","remainder"]}))
])

overlooking_garden_imputer = ColumnTransformer(transformers = [
    ("overlooking_garden_imputer",SimpleImputer(strategy = 'constant',fill_value = -1),["overlooking_garden"])
], remainder = "passthrough")

overlooking_garden_imputation_pipeline = Pipeline(steps = [
    ("overlooking_garden_imputer",overlooking_garden_imputer),
    ("prefix_remover",FunctionTransformer(func = prefix_remover, kw_args = {"prefixes" : ["overlooking_garden_imputer","remainder"]}))
])

overlooking_mainroad_imputer = ColumnTransformer(transformers = [
    ("overlooking_mainroad_imputer",SimpleImputer(strategy = 'constant',fill_value = -1),["overlooking_mainroad"])
], remainder = "passthrough")

overlooking_mainroad_imputation_pipeline = Pipeline(steps = [
    ("overlooking_mainroad_imputer",overlooking_mainroad_imputer),
    ("prefix_remover",FunctionTransformer(func = prefix_remover, kw_args = {"prefixes" : ["overlooking_mainroad_imputer","remainder"]}))
])

overlooking_pool_imputer = ColumnTransformer(transformers = [
    ("overlooking_pool_imputer",SimpleImputer(strategy = 'constant',fill_value = -1),["overlooking_pool"])
], remainder = "passthrough")

overlooking_pool_imputation_pipeline = Pipeline(steps = [
    ("overlooking_pool_imputer",overlooking_pool_imputer),
    ("prefix_remover",FunctionTransformer(func = prefix_remover, kw_args = {"prefixes" : ["overlooking_pool_imputer","remainder"]}))
])

parking_cover_imputer =  ColumnTransformer(transformers = [
    ("parking_cover_imputer",SimpleImputer(strategy = 'constant',fill_value = "No parking"),["parking_cover"])
],remainder = "passthrough")

parking_cover_imputation_pipeline = Pipeline(steps = [
    ("parking_cover_imputer",parking_cover_imputer),
    ("prefix_remover",FunctionTransformer(func = prefix_remover, kw_args = {"prefixes" : ["parking_cover_imputer","remainder"]}))
])

parking_spots_imputer =  ColumnTransformer(transformers = [
    ("parking_spots_imputer",SimpleImputer(strategy = 'constant',fill_value = 0),["parking_spots"])
],remainder = "passthrough")

parking_spots_imputation_pipeline = Pipeline(steps = [
    ("parking_spots_imputer",parking_spots_imputer),
    ("prefix_remover",FunctionTransformer(func = prefix_remover, kw_args = {"prefixes" : ["parking_spots_imputer","remainder"]}))
])

carpet_area_imputer =  ColumnTransformer(transformers = [
    ("carpet_area_imputer",SimpleImputer(strategy = 'constant',fill_value = -1),["carpet_area"])
],remainder = "passthrough")

carpet_area_imputation_pipeline = Pipeline(steps = [
    ("carpet_area_imputer",carpet_area_imputer),
    ("prefix_remover",FunctionTransformer(func = prefix_remover, kw_args = {"prefixes" : ["carpet_area_imputer","remainder"]}))
])

super_area_imputer =  ColumnTransformer(transformers = [
    ("super_area_imputer",SimpleImputer(strategy = 'constant',fill_value = -1),["super_area"])
],remainder = "passthrough")

super_area_imputation_pipeline = Pipeline(steps = [
    ("super_area_imputer",super_area_imputer),
    ("prefix_remover",FunctionTransformer(func = prefix_remover, kw_args = {"prefixes" : ["super_area_imputer","remainder"]}))
])

imputation_pipeline = Pipeline(steps = [
    ("furnishing_imputation_pipeline",furnishing_imputation_pipeline),
    ("floor_num_imputation_pipeline",floor_num_imputation_pipeline),
    ("num_floors_imputation_pipeline",num_floors_imputation_pipeline),
    ("balcony_imputation_pipeline",balcony_imputation_pipeline),
    ("ownership_imputation_pipeline",ownership_imputation_pipeline),
    ("facing_imputation_pipeline",facing_imputation_pipeline),
    ("overlooking_garden_imputation_pipeline",overlooking_garden_imputation_pipeline),
    ("overlooking_mainroad_imputation_pipeline",overlooking_mainroad_imputation_pipeline),
    ("overlooking_pool_imputation_pipeline",overlooking_pool_imputation_pipeline),
    ("parking_cover_imputation_pipeline",parking_cover_imputation_pipeline),
    ("parking_spots_imputation_pipeline",parking_spots_imputation_pipeline),
    ("carpet_area_imputation_pipeline",carpet_area_imputation_pipeline),
    ("super_area_imputation_pipeline",super_area_imputation_pipeline)
])

# Column transformers

transaction_transformer = Pipeline(steps = [
    ("grouper",RareLabelEncoder(tol = 0.1, n_categories = 2, replace_with = "Resale")),
    ("encoder",OneHotEncoder(sparse_output = False,handle_unknown = 'ignore'))
])

def house_size_binner(X):
    columns = X.columns.to_list()

    return (
        X.assign(
               house_size = lambda df: (
                np.select(
                [
                    df.num_bhk.between(1,3,inclusive = "left"),
                    df.num_bhk.between(3,4,inclusive = "left")
                ],
                ["small","normal"],
                default = "big"
               )
            ) 
        )
        .drop(columns = columns)
    )

num_bhk_pipe1 = Pipeline(steps = [
    ("scaler",MinMaxScaler())
])

num_bhk_pipe2 = Pipeline(steps = [
    ("house_size_binner",FunctionTransformer(func = house_size_binner)),
    ("encoder",OrdinalEncoder(categories = [["small","normal","big"]]))
])

num_bhk_transformer = FeatureUnion(transformer_list = [
    ("num_bhk_pipe1",num_bhk_pipe1),
    ("num_bhk_pipe2",num_bhk_pipe2)
])

def bathroom_num_binner(X):
    columns = X.columns.to_list()

    return (
        X.assign(
               bathroom_num = lambda df: (
                np.select(
                [
                    df.bathroom.between(1,3,inclusive = "left"),
                    df.bathroom.between(3,4,inclusive = "left")
                ],
                ["low","medium"],
                default = "high"
              )
            ) 
        )
        .drop(columns = columns)
    )

bathroom_pipe1 = Pipeline(steps = [
    ("scaler",MinMaxScaler())
])

bathroom_pipe2 = Pipeline(steps = [
    ("bathroom_num_binner",FunctionTransformer(func = bathroom_num_binner)),
    ("encoder",OrdinalEncoder(categories = [["low","medium","high"]]))
])

bathroom_transformer = FeatureUnion(transformer_list = [
    ("bathroom_pipe1",bathroom_pipe1),
    ("bathroom_pipe2",bathroom_pipe2)
])

def bathroom_num_binner(X):
    columns = X.columns.to_list()

    return (
        X.assign(
               bathroom_num = lambda df: (
                np.select(
                [
                    df.bathroom.between(1,3,inclusive = "left"),
                    df.bathroom.between(3,4,inclusive = "left")
                ],
                ["low","medium"],
                default = "high"
              )
            ) 
        )
        .drop(columns = columns)
    )

furnishing_pipe1 = Pipeline(steps = [
    ("encoder",OrdinalEncoder(categories = [["Unfurnished","Semi-Furnished","Furnished"]])),
])

furnishing_pipe2 = Pipeline(steps = [
    ("is_unfurnished",FunctionTransformer(func = lambda x: np.where(x == 'Unfurnished',1,0)))
])

furnishing_transformer = FeatureUnion(transformer_list = [
    ("furnishing_pipe1",furnishing_pipe1),
    ("furnishing_pipe2",furnishing_pipe2)
])

def floor_height_binner(X):

    columns = X.columns.to_list()

    return (
        X.assign(
            floor_height = lambda df:(
                    np.select(
                        [
                            (df.floor_num.between(0,3, inclusive = "left")),
                            (df.floor_num.between(3,6, inclusive = "left"))
                        ],
                        ["low","medium"],
                        default = "high"
                    )
            )
        )
        .drop(columns = columns)
    )

def building_height_binner(X):

    columns = X.columns.to_list()

    return (
        X.assign(
            building_height = lambda df:(
                np.select(
                [
                    (df.num_floors.between(0,5, inclusive = "left")),
                    (df.num_floors.between(5,13, inclusive = "left"))
                ],
                ["short","medium"],
                default = "tall"
            )
            )
        )
        .drop(columns = columns)
    )

floor_num_pipe1 = Pipeline(steps = [
    ("scaler",StandardScaler())
])

floor_num_pipe2 = Pipeline(steps = [
    ("floor_height_binner",FunctionTransformer(func = floor_height_binner)),
    ("encoder",OrdinalEncoder(categories = [["low","medium","high"]])),
])

floor_num_transformer = FeatureUnion(transformer_list = [
    ("floor_num_pipe1",floor_num_pipe1),
    ("floor_num_pipe2",floor_num_pipe2)
])

num_floors_pipe1 = Pipeline(steps = [
    ("scaler",StandardScaler())
])

num_floors_pipe2 = Pipeline(steps = [
    ("building_height_binner",FunctionTransformer(func = building_height_binner)),
    ("encoder",OrdinalEncoder(categories = [["short","medium","tall"]]))
])

num_floors_transformer = FeatureUnion(transformer_list = [
    ("num_floors_pipe1",num_floors_pipe1),
    ("num_floors_pipe2",num_floors_pipe2)
])

def city_binner(X):
    
     columns = X.columns.to_list()

     return (
        X.assign(
            city_tier = lambda df:(
                np.where(
                    df.location.isin(["mumbai","gurgaon","new-delhi"]),
                    1,
                    0
                )
            )
        )
        .drop(columns = columns)
    )

location_pipe1 = Pipeline(steps = [
    ("target_encoder", TargetEncoder())
])

location_pipe2 = Pipeline(steps = [
    ("city_binner",FunctionTransformer(func = city_binner))
])

location_transformer = FeatureUnion(transformer_list = [
    ("location_pipe1",location_pipe1),
    ("location_pipe2",location_pipe2)
])

def price_binner(X):

    columns = X.columns.to_list()

    return (
        X.assign(
               price_range = lambda df:(
                np.select(
                    [
                        df.price.between(0,4000,inclusive = "left"),
                        df.price.between(4000,6000,inclusive = "left")
                    ],
                    ["low","medium"],
                    default = "high"
                )
            )
        )
        .drop(columns = columns)
    )

balcony_transformer = Pipeline(steps = [
    ("nearest_integer",FunctionTransformer(func = lambda x : np.round(x))),
    ("scaler",MinMaxScaler())
])

ownership_transformer = Pipeline(steps = [
    ("encoder",OneHotEncoder(sparse_output = False,handle_unknown = 'ignore'))
])

missingindicator_ownership_transformer = Pipeline(steps = [
    ("encoder",OneHotEncoder(drop = 'first',sparse_output = False,handle_unknown = 'ignore'))
])

def direction_binner(X):

     columns = X.columns.to_list()

     return (
        X.assign(
            direction_tier = lambda df:(
                np.where(
                    df.facing.isin(["North - East","North - West"]),
                    1,
                    0
                )
            )
        )
        .drop(columns = columns)
     )

facing_pipe1 = Pipeline(steps = [
    ("encoder",OneHotEncoder(sparse_output = False,handle_unknown = 'ignore'))
])

facing_pipe2 = Pipeline(steps = [
    ("direction_binner",FunctionTransformer(func = direction_binner))
])

facing_transformer = FeatureUnion(transformer_list = [
    ("facing_pipe1",facing_pipe1),
    ("facing_pipe2",facing_pipe2)
])

missingindicator_facing_transformer = Pipeline(steps = [
    ("encoder",OneHotEncoder(drop = 'first',sparse_output = False,handle_unknown = 'ignore'))
])

overlooking_garden_transformer = Pipeline(steps = [
    ("encoder",OneHotEncoder(sparse_output = False,handle_unknown = 'ignore'))
])

overlooking_mainroad_transformer = Pipeline(steps = [
    ("encoder",OneHotEncoder(categories=[[ -1, 0, 1 ]], drop=[-1], sparse_output=False, handle_unknown='ignore'))
])

overlooking_pool_transformer = Pipeline(steps = [
    ("encoder",OneHotEncoder(categories=[[ -1, 0, 1 ]], drop=[-1], sparse_output=False, handle_unknown='ignore'))
])

parking_cover_transformer = Pipeline(steps = [
    ("encoder",OneHotEncoder(sparse_output = False,handle_unknown = 'ignore'))
])

def has_parking(X):

    columns = X.columns.to_list()

    return (
        X.assign(
            has_parking = lambda df:(
                np.select(
                            [
                                df.parking_spots.eq(0),
                                df.parking_spots.eq(1)
                            ],
                            ["no parking","single"],
                            default = "multiple"
                        )
            )
        )
        .drop(columns = columns)
     )

parking_spots_transformer = Pipeline(steps = [
    ("has_parking",FunctionTransformer(func = has_parking)),
    ("encoder",OneHotEncoder(categories = [["multiple","single","no parking"]],drop = [["no parking"]],sparse_output = False,handle_unknown = 'ignore'))
])

def effective_area(X):

    columns = X.columns.to_list()

    return(
        X
        .assign(
            effective_area = lambda df:(
                np.where(
                    df.carpet_area.eq(-1),
                    df.super_area,
                    df.carpet_area
                )
            ),
            carpet_areamissing = lambda df:(
                np.where(
                    df.carpet_area.eq(-1),
                    1,
                    0
                )
            ),
            super_areamissing = lambda df:(
                np.where(
                    df.super_area.eq(-1),
                    1,
                    0
                )
            )
        )
        .drop(columns = columns)
    )

scaler_pipeline = Pipeline(steps = [
    ("log_transformer",FunctionTransformer(func = lambda x: np.log(x))),
    ("scaler",StandardScaler())
])

area_transformer = Pipeline(steps = [
    ("effective_area",FunctionTransformer(func = effective_area)),
    ("scaler_pipeline",ColumnTransformer(transformers = [
        ("scaler_pipeline",scaler_pipeline,["effective_area"])
    ],remainder = "passthrough")),
])

def area_per_room(X):

    columns = X.columns.to_list()

    return(
        X
        .assign(
            area_per_room = lambda df:(
                np.where(
                    df.carpet_area.eq(-1),
                    df.super_area/df.num_bhk,
                    df.carpet_area/df.num_bhk
                )
            )
        )
        .drop(columns = columns)
    )

scaler_pipeline = Pipeline(steps = [
    ("log_transformer",FunctionTransformer(func = lambda x: np.log(x))),
    ("scaler",StandardScaler())
])

area_per_room_transformer = Pipeline(steps = [
    ("area_per_room",FunctionTransformer(func = area_per_room)),
    ("scaler_pipeline",ColumnTransformer(transformers = [
        ("scaler_pipeline",scaler_pipeline,["area_per_room"])
    ],remainder = "passthrough"))
])

def balcony_per_room(X):

    columns = X.columns.to_list()

    return(
        X
        .assign(
            balcony_per_room = lambda df:(
                df.balcony/df.num_bhk
            )
        )
        .drop(columns = columns)
    )

def bathroom_per_room(X):

    columns = X.columns.to_list()

    return(
        X
        .assign(
            bathroom_per_room = lambda df:(
                df.bathroom/df.num_bhk
            )
        )
        .drop(columns = columns)
    )

balcony_per_room_transformer = Pipeline(steps = [
    ("balcony_per_room",FunctionTransformer(func = balcony_per_room))
])

bathroom_per_room_transformer = Pipeline(steps = [
    ("bathroom_per_room",FunctionTransformer(func = bathroom_per_room))
])

column_transformer = ColumnTransformer(transformers = [
    ("transaction_transformer",transaction_transformer,["transaction"]),
    ("num_bhk_transformer",num_bhk_transformer,["num_bhk"]),
    ("bathroom_transformer",bathroom_transformer,["bathroom"]),
    ("furnishing_transformer",furnishing_transformer,["furnishing"]),
    ("floor_num_transformer",floor_num_transformer,["floor_num"]),
    ("num_floors_transformer",num_floors_transformer,["num_floors"]),
    ("location_transformer",location_transformer,["location"]),
    ("balcony_transformer",balcony_transformer,["balcony"]),
    ("ownership_transformer",ownership_transformer,["ownership"]),
    ("missingindicator_ownership_transformer",missingindicator_ownership_transformer,["missingindicator_ownership"]),
    ("facing_transformer",facing_transformer,["facing"]),
    ("missingindicator_facing_transformer",missingindicator_ownership_transformer,["missingindicator_facing"]),
    ("overlooking_garden_transformer",overlooking_garden_transformer,["overlooking_garden"]),
    ("overlooking_mainroad_transformer",overlooking_mainroad_transformer,["overlooking_mainroad"]),
    ("overlooking_pool_transformer",overlooking_pool_transformer,["overlooking_pool"]),
    ("parking_cover_transformer",parking_cover_transformer,["parking_cover"]),
    ("parking_spots_transformer",parking_spots_transformer,["parking_spots"]),
    ("area_transformer",area_transformer,["carpet_area","super_area"]),
    ("area_per_room_transformer",area_per_room_transformer,["carpet_area","super_area","num_bhk"]),
    ("balcony_per_room_transformer",balcony_per_room_transformer,["balcony","num_bhk"]),
    ("bathroom_per_room_transformer",bathroom_per_room_transformer,["bathroom","num_bhk"])
],remainder = 'passthrough')

# Final preprocessor

feature_preprocessor = Pipeline(steps = [
    ("imputation_pipeline",imputation_pipeline),
    ("column_transformer",column_transformer)
])

# Correlated feature selector
drop_correlated = SmartCorrelatedSelection(estimator = RandomForestRegressor(), scoring = 'r2')

# Feature selector
feature_selector = SelectBySingleFeaturePerformance(estimator = RandomForestRegressor(),scoring = 'r2',threshold = 0.01)

# Feature engineering pipeline
feature_engineering_pipeline = Pipeline(steps = [
    ("feature_preprocessor",feature_preprocessor),
    ("drop_correlated",drop_correlated),
    ("feature_selector",feature_selector)
])

In [84]:
X_train_preprocessed = feature_engineering_pipeline.fit_transform(X_train,y_train)
X_val_preprocessed = feature_engineering_pipeline.transform(X_val)
X_test_preprocessed = feature_engineering_pipeline.transform(X_test)

## 5. Hyperparameter Tuning and Experimentation Tracking 

In [19]:
mlflow.is_tracking_uri_set()

False

In [54]:
def objective(trial):
    
    with mlflow.start_run(nested = True):
    
            regressor_name = trial.suggest_categorical("regressor", [
                "LinearRegression",
                "RidgeRegression",
                "LassoRegression",
                "KNeighborsRegressor",
                "RandomForestRegressor",
                "AdaBoostRegressor",
                "GradientBoostingRegressor",
                "XGBRegressor",
            ])
        
            if regressor_name == "LinearRegression":
                fit_intercept = trial.suggest_categorical("fit_intercept", [True, False])
                regressor = LinearRegression(
                    fit_intercept=fit_intercept,
                    n_jobs=-1
                )
                mlflow.log_param("fit_intercept",fit_intercept)
        
            elif regressor_name == "RidgeRegression":
                alpha = trial.suggest_float("alpha", 0.0001, 10.0, log=True)
                fit_intercept = trial.suggest_categorical("fit_intercept", [True, False])
                regressor = Ridge(
                    alpha=alpha,
                    fit_intercept=fit_intercept
                )
                mlflow.log_params({"alpha": alpha, "fit_intercept": fit_intercept})
        
            elif regressor_name == "LassoRegression":
                alpha = trial.suggest_float("alpha", 0.0001, 1.0, log=True)
                fit_intercept = trial.suggest_categorical("fit_intercept", [True, False])
                regressor = Lasso(
                    alpha=alpha,
                    fit_intercept=fit_intercept
                )
                mlflow.log_params({"alpha": alpha, "fit_intercept": fit_intercept})
        
            elif regressor_name == "KNeighborsRegressor":
                n_neighbors = trial.suggest_int("n_neighbors", 2, 30)
                weights = trial.suggest_categorical("weights", ["uniform", "distance"])
                p = trial.suggest_int("p", 1, 2)  
                regressor = KNeighborsRegressor(
                    n_neighbors=n_neighbors,
                    weights=weights,
                    p=p,
                    n_jobs=-1
                )
                mlflow.log_params({"n_neighbors": n_neighbors, "weights": weights, "p": p})
                
            elif regressor_name == "RandomForestRegressor":
                n_estimators = trial.suggest_int("n_estimators", 50, 300)
                max_depth = trial.suggest_int("max_depth", 2, 20)
                min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
                max_features = trial.suggest_categorical("max_features", ["sqrt", "log2", None])
                regressor = RandomForestRegressor(
                    n_estimators=n_estimators,
                    max_depth=max_depth,
                    min_samples_split=min_samples_split,
                    max_features=max_features,
                    n_jobs=-1
                )
                mlflow.log_params({"n_estimators": n_estimators, "max_depth": max_depth, "min_samples_split": min_samples_split, "max_features": max_features})
        
            elif regressor_name == "AdaBoostRegressor":
                n_estimators = trial.suggest_int("n_estimators", 50, 300)
                learning_rate = trial.suggest_float("learning_rate", 0.001, 2.0, log=True)
                loss = trial.suggest_categorical("loss", ["linear", "square", "exponential"])
                regressor = AdaBoostRegressor(
                    n_estimators=n_estimators,
                    learning_rate=learning_rate,
                    loss=loss
                )
                mlflow.log_params({"n_estimators": n_estimators, "learning_rate": learning_rate, "loss": loss})
        
            elif regressor_name == "GradientBoostingRegressor":
                n_estimators = trial.suggest_int("n_estimators", 50, 300)
                subsample = trial.suggest_float("subsample", 0.5, 1.0)
                learning_rate = trial.suggest_float("learning_rate", 0.001, 2.0, log=True)
                min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
                max_depth = trial.suggest_int("max_depth", 2, 10)
                max_features = trial.suggest_categorical("max_features", ["sqrt", "log2", None])
                regressor = GradientBoostingRegressor(
                    n_estimators=n_estimators,
                    subsample=subsample,
                    learning_rate=learning_rate,
                    min_samples_split=min_samples_split,
                    max_depth=max_depth,
                    max_features=max_features
                )
                mlflow.log_params({"n_estimators": n_estimators, "subsample": subsample, "learning_rate": learning_rate, "min_samples_split": min_samples_split, "max_depth": max_depth, "max_features": max_features})
        
            elif regressor_name == "XGBRegressor":
                n_estimators = trial.suggest_int("n_estimators", 50, 300)
                learning_rate = trial.suggest_float("learning_rate", 0.001, 0.3, log=True)
                max_depth = trial.suggest_int("max_depth", 2, 10)
                subsample = trial.suggest_float("subsample", 0.5, 1.0)
                colsample_bytree = trial.suggest_float("colsample_bytree", 0.5, 1.0)
                regressor = XGBRegressor(
                    n_estimators=n_estimators,
                    learning_rate=learning_rate,
                    max_depth=max_depth,
                    subsample=subsample,
                    colsample_bytree=colsample_bytree,
                    n_jobs=-1
                )
                mlflow.log_params({"n_estimators": n_estimators, "learning_rate": learning_rate, "max_depth": max_depth, "subsample": subsample, "colsample_bytree": colsample_bytree})

            mlflow.log_param("regressor",regressor_name)
        
            target_transformer_name = trial.suggest_categorical(
                "target_transformer",
                ["log","cbrt","sqrt","same"]
            )
            mlflow.log_param("target_transformer",target_transformer_name)
        
            if target_transformer_name == "log":
                transformer = FunctionTransformer(
                    func = np.log,
                    inverse_func = np.exp
                )
                
            elif target_transformer_name == "cbrt":
                transformer = FunctionTransformer(
                    func = np.cbrt,
                    inverse_func = lambda x:x**3
                )
        
            elif target_transformer_name == "sqrt":
                transformer = FunctionTransformer(
                    func = np.sqrt,
                    inverse_func = lambda x:x**2
                )
        
            elif target_transformer_name == "same":
                transformer = FunctionTransformer(
                    func = lambda x:x,
                    inverse_func = lambda x:x
                )
        
            model_pipeline = Pipeline(steps=[
                ("model", TransformedTargetRegressor(
                    regressor=regressor,
                    transformer=transformer
                ))
            ])
        
            mae = 0
            rmse = 0
            r2= 0
            for i in range(3):
                model_pipeline.fit(X_train_preprocessed, y_train)
        
                train_mae = mean_absolute_error(y_train,model_pipeline.predict(X_train_preprocessed))
                val_mae = mean_absolute_error(y_val,model_pipeline.predict(X_val_preprocessed))
                
                val_r2_score = r2_score(y_val,model_pipeline.predict(X_val_preprocessed))
                val_rmse = root_mean_squared_error(y_val,model_pipeline.predict(X_val_preprocessed))
                
                overfit_penalty = abs(val_mae-train_mae)
                mae += val_mae + overfit_penalty
                r2 += val_r2_score
                rmse += val_rmse
                
            avg_mae = mae/3
            avg_r2 = r2/3
            avg_rmse = rmse/3
        
            mlflow.log_metric("avg_mae",avg_mae)
            mlflow.log_metric("avg_r2_score",avg_r2)
            mlflow.log_metric("avg_rmse",avg_rmse)
        
            return avg_mae

In [56]:
mlflow.set_tracking_uri('http://127.0.0.1:5000')
mlflow.set_experiment("House Price Prediction")

with mlflow.start_run(run_name = "No group aggregate imputer") as run:
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials= 120)

    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_value",study.best_value) 

[I 2025-08-16 01:04:13,495] A new study created in memory with name: no-name-ecf3d03b-4bc6-4b7f-94ed-ead428b7f8ba
[I 2025-08-16 01:04:14,621] Trial 0 finished with value: 0.1985037869396957 and parameters: {'regressor': 'RidgeRegression', 'alpha': 0.00629170534393477, 'fit_intercept': True, 'target_transformer': 'log'}. Best is trial 0 with value: 0.1985037869396957.


🏃 View run calm-bird-866 at: http://127.0.0.1:5000/#/experiments/812027865869456740/runs/3dc53fbd70524acb837d25adaff645f0
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/812027865869456740


[I 2025-08-16 01:07:50,356] Trial 1 finished with value: 0.35530399987804256 and parameters: {'regressor': 'KNeighborsRegressor', 'n_neighbors': 9, 'weights': 'distance', 'p': 1, 'target_transformer': 'log'}. Best is trial 0 with value: 0.1985037869396957.


🏃 View run monumental-lark-159 at: http://127.0.0.1:5000/#/experiments/812027865869456740/runs/750e73d5b94f4f56b0e89d6e505bc1b2
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/812027865869456740


[I 2025-08-16 01:10:53,631] Trial 2 finished with value: 0.24155198207137665 and parameters: {'regressor': 'AdaBoostRegressor', 'n_estimators': 197, 'learning_rate': 0.062184307874534744, 'loss': 'exponential', 'target_transformer': 'same'}. Best is trial 0 with value: 0.1985037869396957.


...

🏃 View run gregarious-snail-124 at: http://127.0.0.1:5000/#/experiments/812027865869456740/runs/b5766e0dbf044592b30965a507e2aa13
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/812027865869456740


[I 2025-08-16 03:13:52,501] Trial 118 finished with value: 0.3558246890646873 and parameters: {'regressor': 'KNeighborsRegressor', 'n_neighbors': 8, 'weights': 'distance', 'p': 1, 'target_transformer': 'log'}. Best is trial 111 with value: 0.17108495337792665.


🏃 View run delicate-finch-572 at: http://127.0.0.1:5000/#/experiments/812027865869456740/runs/770bdb8348834e93b70f24d269f15d5d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/812027865869456740


[I 2025-08-16 03:15:37,857] Trial 119 finished with value: 0.17523730111101363 and parameters: {'regressor': 'GradientBoostingRegressor', 'n_estimators': 294, 'subsample': 0.71867536700462, 'learning_rate': 0.015879437730595202, 'min_samples_split': 2, 'max_depth': 6, 'max_features': 'log2', 'target_transformer': 'log'}. Best is trial 111 with value: 0.17108495337792665.


🏃 View run sedate-lynx-190 at: http://127.0.0.1:5000/#/experiments/812027865869456740/runs/f66be4f43b31408aa70feed7daffee32
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/812027865869456740
🏃 View run No group aggregate imputer at: http://127.0.0.1:5000/#/experiments/812027865869456740/runs/9af58e5ef8c6456fa88e04cfe574febe
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/812027865869456740


## Best Model after Experimentaion Tracking

- Group Aggregate Imputer instead of Simple Imputer in `furnishing`,`num_floors` and `balcony`.
- Dropping correlated features
- Feature Selection bringing no of columns down to `18`.
- The model params are  
| regressor   | target_transformer | subsample   | n_estimators | max_depth | learning_rate | colsample_bytree |
|-------------|--------------------|-------------|--------------|-----------|---------------|------------------|
| XGBRegressor| log                | 0.9776623167| 189          | 3         | 0.1459533351  | 0.8668934622     |
   |

## 6. Final Evaluation on Test Model

In [96]:
regressor = XGBRegressor(
    n_estimators = 189,
    max_depth = 3,
    learning_rate = 0.1459533351,
    subsample = 0.9776623167,
    colsample_bytree = 0.8668934622
)

transformer = FunctionTransformer(
                    func = np.log,
                    inverse_func = np.exp
                )

model_pipeline = Pipeline(steps=[
                ("model", TransformedTargetRegressor(
                    regressor=regressor,
                    transformer=transformer
                ))
            ])

model_pipeline.fit(X_train_preprocessed,y_train)
y_train_pred = model_pipeline.predict(X_train_preprocessed)
y_val_pred = model_pipeline.predict(X_val_preprocessed)
y_test_pred = model_pipeline.predict(X_test_preprocessed)

train_mae = mean_absolute_error(y_train,y_train_pred)
val_mae = mean_absolute_error(y_val,y_val_pred)
test_mae = mean_absolute_error(y_test,y_test_pred)

train_r2 = r2_score(y_train,y_train_pred)
val_r2 = r2_score(y_val,y_val_pred)
test_r2 = r2_score(y_test,y_test_pred)

train_rmse = root_mean_squared_error(y_train,y_train_pred)
val_rmse = root_mean_squared_error(y_val,y_val_pred)
test_rmse = root_mean_squared_error(y_test,y_test_pred)

results_df = pd.DataFrame({
                            "data": ["train","val","test"],
                            "mae": [train_mae,val_mae,test_mae],
                            "r2_score": [train_r2,val_r2,test_r2],
                            "rmse": [train_rmse,val_rmse,test_rmse]
}).set_index("data")

results_df

,mae,r2_score,rmse
data,,,
train,0.166844,0.780897,0.262478
val,0.171843,0.759893,0.279042
test,0.175137,0.761845,0.271955
